# PostGIS on Greenplum Database
## 8.1 System preparation
### 8.1.1 Set system variables, connection string, Greenplum Database auto stats, etc.

In [5]:
import os, re
from IPython.display import display_html

import pygments.lexers
from pygments import highlight
from pygments.formatters import HtmlFormatter

CONNECTION_STRING = os.getenv('AWSGPDBCONN')

cs = re.match('^postgresql:\/\/(\S+):(\S+)@(\S+):(\S+)\/(\S+)$', CONNECTION_STRING)

DB_USER   = cs.group(1)
DB_PWD    = cs.group(2)
DB_SERVER = cs.group(3)
DB_PORT   = cs.group(4)
DB_NAME   = cs.group(5)

%reload_ext sql
%sql $CONNECTION_STRING

'Connected: gpadmin@gpadmin'

In [6]:
%%sql $DB_USER@$DB_SERVER
SHOW gp_autostats_mode;
ALTER DATABASE gpadmin SET gp_autostats_mode TO 'NONE';
SHOW gp_autostats_mode;

1 rows affected.
Done.
1 rows affected.


gp_autostats_mode
NONE


### 8.1.2 Prepare AWS System and setup awscli library via pip

In [11]:
shfilecode = !pygmentize -f html -O full,style=friendly -l shell script/1-1-system-prepare.sh
display_html('\n'.join(shfilecode), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 #!/bin/bash 

 cd $HOME 

curl -O https://bootstrap.pypa.io/get-pip.py
python $HOME /get-pip.py --force-reinstall
rm $HOME /get-pip.py

mv $HOME /.bash_profile $HOME /.bash_profile.old

cat >> $HOME /.bash_profile << 'EOF' 
 # .bash_profile 

 # Get the aliases and functions 
 if [ -f ~/.bashrc ]; then 
 . ~/.bashrc 
 fi 

 # User specific environment and startup programs 

 export PATH=$HOME/.local/bin:$HOME/bin:$PATH 

 EOF 

chmod 644 $HOME /.bash_profile
 source $HOME /.bash_profile

pip install --upgrade --force-reinstall pip
pip install --upgrade --force-reinstall boto3
pip install --upgrade --force-reinstall awscli

 echo "***********************" 
 echo "* Get AWS CLI version *" 
 echo "***********************" 
aws --version

In [12]:
!ssh-keygen -R $DB_SERVER
!ssh-keyscan $DB_SERVER >> ~/.ssh/known_hosts
!scp -i ~/.ssh/aws-gp.pem script/1-1-system-prepare.sh $DB_USER@$DB_SERVER:system-prepare.sh
!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER 'chmod +x ./system-prepare.sh'
!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER 'sudo ./system-prepare.sh'

Host ec2-3-8-137-134.eu-west-2.compute.amazonaws.com not found in /root/.ssh/known_hosts
# ec2-3-8-137-134.eu-west-2.compute.amazonaws.com:22 SSH-2.0-OpenSSH_7.4
# ec2-3-8-137-134.eu-west-2.compute.amazonaws.com:22 SSH-2.0-OpenSSH_7.4
# ec2-3-8-137-134.eu-west-2.compute.amazonaws.com:22 SSH-2.0-OpenSSH_7.4
1-1-system-prepare.sh                         100%  712    76.7KB/s   00:00    
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1734k  100 1734k    0     0  11.6M      0 --:--:-- --:--:-- --:--:-- 11.6M
DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
  Found existing installat

  Found existing installation: jmespath 0.9.4
    Uninstalling jmespath-0.9.4:
      Successfully uninstalled jmespath-0.9.4
  Found existing installation: docutils 0.15.2
    Uninstalling docutils-0.15.2:
      Successfully uninstalled docutils-0.15.2
  Found existing installation: six 1.12.0
    Uninstalling six-1.12.0:
      Successfully uninstalled six-1.12.0
  Found existing installation: python-dateutil 2.8.0
    Uninstalling python-dateutil-2.8.0:
      Successfully uninstalled python-dateutil-2.8.0
  Found existing installation: urllib3 1.25.6
    Uninstalling urllib3-1.25.6:
      Successfully uninstalled urllib3-1.25.6
  Found existing installation: botocore 1.12.250
    Uninstalling botocore-1.12.250:
      Successfully uninstalled botocore-1.12.250
  Found existing installation: PyYAML 3.10
***********************
* Get AWS CLI version *
***********************
ERROR: Cannot uninstall 'PyYAML'. It is a distutils installed project and thus we cannot accurately determine whic

### 8.1.3 Provide AWS Access Key ID & Secret Access Key

In [13]:
shfilecode = !pygmentize -f html -O full,style=friendly -l bash script/1-2-aws-configure.sh
display_html('\n'.join(shfilecode), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 #!/bin/bash 

 if [ -d $HOME /.aws ] ; then rm -rf $HOME /.aws ; fi 

mkdir $HOME /.aws

 echo "[default] 
 aws_access_key_id=" $1 " 
 aws_secret_access_key=" $2 "" > $HOME /.aws/credentials

 echo "[default] 
 region=eu-west-2 
 output=json" > $HOME /.aws/config

 #test aws config 
 if [ " $( /usr/local/bin/aws s3 ls s3://amazon-reviews-pds/tsv/ | awk '{print $4}' | awk 'END {print NR}' ) " -eq "0" ] ; 
 then echo "Error on AWS S3 Configuration setup" ; 
 else echo "AWS S3 Configuration setup correctly" ; 
 fi

In [14]:
import getpass

!scp -i ~/.ssh/aws-gp.pem script/1-2-aws-configure.sh $DB_USER@$DB_SERVER:aws-configure.sh
!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER 'chmod +x ./aws-configure.sh'

cmd = 'sudo ./aws-configure.sh ' 
cmd = cmd + getpass.getpass("AWS Access Key ID [None]:") 
cmd = cmd + ' ' + getpass.getpass("AWS Secret Access Key [None]:")

!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER $cmd

1-2-aws-configure.sh                          100%  484    79.2KB/s   00:00    
AWS Access Key ID [None]:········
AWS Secret Access Key [None]:········
AWS S3 Configuration setup correctly


### 8.1.4 Create Greenplum Database Schema and Tables for Demo

In [7]:
sqlfilecode = !pygmentize -f html -O full,style=friendly -l postgres script/8-0-db-maintenance.sql
display_html('\n'.join(sqlfilecode), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 DROP SCHEMA IF EXISTS demo CASCADE ; 
 VACUUM FULL VERBOSE ;

In [8]:
query = !cat script/8-0-db-maintenance.sql
%sql $DB_USER@$DB_SERVER {''.join(query)}

Done.
Done.


[]

In [9]:
sqlfilecode = !pygmentize -f html -O full,style=friendly -l postgres script/8-1-create-schema-tables.sql
display_html('\n'.join(sqlfilecode), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 CREATE SCHEMA demo ; 

 CREATE TABLE demo . uber_mov_ldn ( 
 qoy INTEGER , 
 sourceid INTEGER , 
 dstid INTEGER , 
 hod INTEGER , 
 mean_travel_time NUMERIC , 
 standard_deviation_travel_time NUMERIC , 
 geometric_mean_travel_time NUMERIC , 
 geometric_standard_deviation_travel_time NUMERIC ) 
 DISTRIBUTED BY ( sourceid , dstid ) 
 PARTITION BY LIST ( qoy ) 
 SUBPARTITION BY RANGE ( hod ) 
 SUBPARTITION TEMPLATE ( 
 START ( 0 ) END ( 23 ) EVERY ( 1 ), 
 DEFAULT SUBPARTITION OTHERHOD ) 
 ( VALUES ( 201601 , 201602 , 201603 , 201604 , 201701 , 201702 , 201703 , 201704 , 201801 ), DEFAULT PARTITION OTHERQOY ); 
 
 DROP TABLE IF EXISTS demo . postcode_lookup ; 
 CREATE TABLE demo . postcode_lookup ( 
 pcd7 TEXT , 
 pcd8 TEXT , 
 pcds TEXT , 
 dointr TEXT , 
 doterm TEXT , 
 usertype TEXT , 
 oa11cd TEXT , 
 lsoa11cd TEXT , 
 msoa11cd TEXT , 
 ladcd TEXT , 
 lsoa11nm TEXT , 
 msoa11nm TEXT , 
 ladnm TEXT , 
 ladnmw TEXT ) 
 DISTRIBUTED RANDOMLY ; 

 DROP TABLE IF EXISTS demo . colour_gradient ; 
 CREATE TABLE demo . colour_gradient ( 
 id SMALLINT , 
 min INT , 
 max INT , 
 RGB TEXT ) 
 DISTRIBUTED BY ( id ); 

 INSERT INTO demo . colour_gradient ( id , min , max , RGB ) 
 VALUES ( 01 , 00 , 300 , '04C8AC' ), 
 ( 02 , 300 , 600 , '03B7A5' ), 
 ( 03 , 600 , 900 , '03A79E' ), 
 ( 04 , 900 , 1200 , '039698' ), 
 ( 05 , 1200 , 1500 , '028691' ), 
 ( 06 , 1500 , 1800 , '02758A' ), 
 ( 07 , 1800 , 2100 , '026584' ), 
 ( 08 , 2100 , 2400 , '01557D' ), 
 ( 09 , 2400 , 2700 , '014476' ), 
 ( 10 , 2700 , 3000 , '013470' ), 
 ( 11 , 3000 , 3300 , '002369' ), 
 ( 12 , 3300 , 3600 , '001362' ), 
 ( 13 , 3600 , 10000 , '00035C' );

In [10]:
query = !cat script/8-1-create-schema-tables.sql
%sql $DB_USER@$DB_SERVER {''.join(query)}

Done.
Done.
Done.
Done.
Done.
Done.
13 rows affected.


[]

In [ ]:
sqlfilecode = !pygmentize -f html -O full,style=friendly -l postgres script/8-2-create-london-lsoa-table.sql
display_html('\n'.join(sqlfilecode), raw=True)

### 8.1.5 Copy source files from AWS S3

In [ ]:
shfilecode = !pygmentize -f html -O full,style=friendly -l bash script/8-3-copy-data-from-s3.sh
display_html('\n'.join(shfilecode), raw=True)

## 8.2 PostGIS

In [ ]:
!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER '